<a href="https://colab.research.google.com/github/AmiMunshi/NaturalLanguageProcessing/blob/main/Basic_Question_Answer_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.8 MB/s eta 0:00:00


In [ ]:
!pip install bert-serving-server bert-serving-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=a1fb7c1343e9dbff9076889fc864c702ae4b08e69d00da950cea747f126cd1d1
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


### Using naive method to retrieve the answer for the given question

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv('QA.csv')

In [ ]:
data

,Question,Answer
0,What is capital of Maharashtra?,Mumbai
1,What is population of Mumbai?,1.25 Crore
2,What was Mumbai earlier called?,Bombay
3,In which place Asia's largest slum located?,Dharavi
4,Mumbai is made up of which seven different isl...,"Colaba, Mazagaon, Little Colaba, Parel, Mahim,..."
5,Which is Mumbai's first five star hotel?,Taj Mahal Palace


In [ ]:
#Using iterrow() to iterate over a column of data frame
for q in data.iterrows():
    print(q[1][0])

What is capital of Maharashtra?
What is population of Mumbai?
What was Mumbai earlier called?
In which place Asia's largest slum located?
Mumbai is made up of which seven different islands
Which is Mumbai's first five star hotel?


In [ ]:
#iterating over a column of the dataframe
for i in range(data.shape[0]):
  print(data["Question"][i])

What is capital of Maharashtra?
What is population of Mumbai?
What was Mumbai earlier called?
In which place Asia's largest slum located?
Mumbai is made up of which seven different islands
Which is Mumbai's first five star hotel?


In [ ]:
#Function to count number of words matching in two strings
def string_compare(q1,q2):
  q1=q1.lower()
  q1= q1.split(" ")
  q2=q2.lower()
  q2= q2.split(" ")
  same_count= len(list(set(q1)&set(q2)))
  return same_count

In [ ]:
#Checking to see if function is working appropriately
string_compare(data["Question"][0], "What is capital of Maharashtra?")

5

In [ ]:
#Function to compare the question with each question in data frame
#The function will return number of matches, similar question in the dataset and the correspoding answer
def ask(curiosity):
  count_similar=[]
  data = pd.read_csv('QA.csv')

  #Run the for loop for number of question times
  for i in range(data.shape[0]):
    #Call string_compare function, append the count returned by the function to the list
    count_similar.append(string_compare(data["Question"][i], curiosity))

    #List index of the maximum count will be the index of question in the dataset
    max_count= max(count_similar)
    n=count_similar.index(max_count)
    similar_question=data["Question"][n]
    suggested_answer=data["Answer"][n]
  return count_similar, similar_question, suggested_answer


In [ ]:
question=input("Ask you question: ")
count, q, ans= ask(question)
print(ans)

Ask you question: name of maharastra
Mumbai


### Using ratio function from Levenshtein package to find the similarity in two strings

In [ ]:
import Levenshtein as LS
import pandas as pd

In [ ]:
#Find similarity ratio

def give_ratio(curiosity):
  similar_ratio=[]
  data = pd.read_csv('QA.csv')

  #Run the for loop for number of question times
  for i in range(data.shape[0]):
    #Call string_compare function, append the count returned by the function to the list
    similar_ratio.append(LS.ratio(data["Question"][i], curiosity))

    #List index of the maximum count will be the index of question in the dataset
    max_ratio= max(similar_ratio)
    n=similar_ratio.index(max_ratio)
    similar_question=data["Question"][n]
    suggested_answer=data["Answer"][n]
  return max_ratio, similar_question, suggested_answer

In [ ]:
question=input("Ask you question: ")
ratio, q, ans= give_ratio(question)
print(ans, ratio)

In [ ]:
from bert_serving.client import BertClient
import numpy as np

def encode_questions():
    bc = BertClient()
    questions = data["Question"].values.tolist()
    print("Questions count", len(questions))
    print("Start to calculate encoder....")
    questions_encoder = bc.encode(questions)
    np.save("questions", questions_encoder)
    questions_encoder_len = np.sqrt(
        np.sum(questions_encoder * questions_encoder, axis=1)
    )
    np.save("questions_len", questions_encoder_len)
    print("Encoder ready")

encode_questions()

Ref:

https://typesense.org/learn/fuzzy-string-matching-python/

https://intersog.com/blog/the-basics-of-qa-systems-from-a-single-function-to-a-pre-trained-nlp-model-using-python/